In [1]:
# from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

In [3]:
from env import host, user, password

def get_db_url(database, host=host, user=user, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'

In [4]:
url = get_db_url("grocery_db")

sql = '''
select * from grocery_customers
'''

df = pd.read_sql(sql,url, index_col="customer_id")

In [5]:
df

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
customer_id,,,,,,,,
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185
...,...,...,...,...,...,...,...,...
435,1,3,29703,12051,16027,13135,182,2204
436,1,3,39228,1431,764,4510,93,2346
437,2,3,14531,15488,30243,437,14841,1867


In [6]:
# Cluster on all or a selected subset of features
grocery_milk_fresh = df[['Grocery', 'Milk', 'Fresh']]

In [7]:
grocery_milk_fresh

,Grocery,Milk,Fresh
customer_id,,,
0,7561,9656,12669
1,9568,9810,7057
2,7684,8808,6353
3,4221,1196,13265
4,7198,5410,22615
...,...,...,...
435,16027,12051,29703
436,764,1431,39228
437,30243,15488,14531


### Use MinMaxScaler to scale the newly created dataframe.

In [8]:
# Create the scaler object.
scaler = MinMaxScaler()

# Fit the scaler object on the data from grocery_milk_fresh dataframe.
scaler.fit(grocery_milk_fresh)

# Use the fitted scaler object.
grocery_milk_fresh = scaler.transform(grocery_milk_fresh)

In [9]:
grocery_milk_fresh

array([[0.08146416, 0.13072723, 0.11294004],
       [0.10309667, 0.13282409, 0.06289903],
       [0.08278992, 0.11918086, 0.05662161],
       ...,
       [0.32594285, 0.21013575, 0.1295431 ],
       [0.02402535, 0.02622442, 0.091727  ],
       [0.02702178, 0.02237109, 0.02482434]])

### Construct a DBSCAN object that requires a minimum of 20 data points in a neighborhood of radius 0.1 to be considered a core point.

In [10]:
# Create the DBSCAN object.
dbsc = DBSCAN(eps = .10, min_samples = 20)

# Fit the DBSCAN object on the grocery-milk-fresh
dbsc.fit(grocery_milk_fresh)

DBSCAN(eps=0.1, min_samples=20)

In [15]:
# Add the scaled dataframe back to the original dataframe.
columns = ['Grocery','Milk','Fresh']
scaled_columns = ['Scaled_' + column for column in columns]

# Create a copy of the original dataframe for a quick and easy restart.
original_df = df.copy()

# Create a dataframe containing the scaled values.
scaled_df = pd.DataFrame(grocery_milk_fresh, columns=scaled_columns)

# Merge the scaled dataframe to the original dataframe.
df = df.merge(scaled_df, on=df.index)
df = df.drop(columns=['key_0'])

In [16]:
df

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Scaled_Grocery,Scaled_Milk,Scaled_Fresh
0,2,3,12669,9656,7561,214,2674,1338,0.081464,0.130727,0.112940
1,2,3,7057,9810,9568,1762,3293,1776,0.103097,0.132824,0.062899
2,2,3,6353,8808,7684,2405,3516,7844,0.082790,0.119181,0.056622
3,1,3,13265,1196,4221,6404,507,1788,0.045464,0.015536,0.118254
4,2,3,22615,5410,7198,3915,1777,5185,0.077552,0.072914,0.201626
...,...,...,...,...,...,...,...,...,...,...,...
435,1,3,29703,12051,16027,13135,182,2204,0.172715,0.163338,0.264829
436,1,3,39228,1431,764,4510,93,2346,0.008202,0.018736,0.349761
437,2,3,14531,15488,30243,437,14841,1867,0.325943,0.210136,0.129543
438,1,3,10290,1981,2232,1038,168,2125,0.024025,0.026224,0.091727


### 